In [1]:
import solopy
import ccdproc
from pathlib import Path
import logging

In [2]:
subdir = "2025_0719" # observation date (=subdirectory name)

# Data directory
DATADIR = Path("../solo-data")

# Lv0 data directory
LV0DIR = DATADIR/"Lv0"
LV0_subdir = LV0DIR / subdir

# Lv1 data directory
LV1DIR = DATADIR/"Lv1"
LV1_subdir = LV1DIR / subdir
LV1_subdir.mkdir(parents=True, exist_ok=True)

# Master calibration files directory
MASTERDIR = DATADIR / "calibration_files"
MASTERDIR.mkdir(parents=True, exist_ok=True)

# Log directory
LOGDIR = DATADIR/"log"
LOGDIR.mkdir(parents=True, exist_ok=True)

fpath_log_lv0 = LOGDIR/f'lv0_{subdir}.log' # log file path for Lv0
# if fpath_log_lv0.exists():
#     fpath_log_lv0.unlink() # remove existing log file
    
fpath_log_lv1 = LOGDIR/f'lv1_{subdir}.log' # log file path for Lv1
# if fpath_log_lv1.exists():
#     fpath_log_lv1.unlink() # remove existing log file

fpath_log_comb = LOGDIR/f'combmaster_{subdir}.log' # log file path for CombMaster
# if fpath_log_comb.exists():
#     fpath_log_comb.unlink() # remove existing log file

In [3]:
# Logger
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[
                        logging.StreamHandler(),
                        logging.FileHandler(LOGDIR/"solopy-example.log")
                    ])
logger = logging.getLogger(__name__)

In [12]:
### Lv0: Header Update
logger.info("--- Step 1: Lv0 Header Update ---")

lv0 = solopy.Lv0(log_file=fpath_log_lv0)

# Collect all Lv0 fits files
all_lv0_fits = ccdproc.ImageFileCollection(LV0_subdir, glob_exclude="*.bz2", glob_include="*.fits").files_filtered(include_path=True)

if not all_lv0_fits:
    raise FileNotFoundError(f"No .fits files found in {LV0_subdir}.")

for fpath in all_lv0_fits:
    lv0.update_header(fpath) # Update header in-place

2025-11-03 16:40:42,116 [INFO] --- Step 1: Lv0 Header Update ---
2025-11-03 16:40:42,662 [INFO] Updated LV0 header: bias_001.fits
2025-11-03 16:40:42,662 [INFO] Updated LV0 header: bias_001.fits
2025-11-03 16:40:42,662 [INFO] Updated LV0 header: bias_001.fits
2025-11-03 16:40:42,749 [INFO] Updated LV0 header: bias_002.fits
2025-11-03 16:40:42,749 [INFO] Updated LV0 header: bias_002.fits
2025-11-03 16:40:42,749 [INFO] Updated LV0 header: bias_002.fits
2025-11-03 16:40:42,834 [INFO] Updated LV0 header: bias_003.fits
2025-11-03 16:40:42,834 [INFO] Updated LV0 header: bias_003.fits
2025-11-03 16:40:42,834 [INFO] Updated LV0 header: bias_003.fits
2025-11-03 16:40:42,916 [INFO] Updated LV0 header: bias_004.fits
2025-11-03 16:40:42,916 [INFO] Updated LV0 header: bias_004.fits
2025-11-03 16:40:42,916 [INFO] Updated LV0 header: bias_004.fits
2025-11-03 16:40:43,001 [INFO] Updated LV0 header: bias_005.fits
2025-11-03 16:40:43,001 [INFO] Updated LV0 header: bias_005.fits
2025-11-03 16:40:43,001 [

In [5]:
logger.info("--- Step 2:  CombMaster Frame Generation ---")
comb = solopy.CombMaster(log_file=fpath_log_comb)

# Collect all Lv0 fits files
all_lv0_fits = ccdproc.ImageFileCollection(LV0_subdir, glob_exclude="*.bz2", glob_include="*.fits")

# Master bias
bias_files = all_lv0_fits.files_filtered(IMAGETYP="BIAS", include_path=True)
if bias_files:
    comb.make_mbias(bias_files, MASTERDIR)
    
# Master dark
dark_files = all_lv0_fits.files_filtered(IMAGETYP="DARK", include_path=True)
if dark_files:
    comb.make_mdark(dark_files, MASTERDIR, key_exptime='EXPTIME')

2025-11-03 16:10:38,952 [INFO] --- Step 2:  CombMaster Frame Generation ---
2025-11-03 16:10:39,775 [INFO] Starting master bias combination...
2025-11-03 16:10:39,776 [INFO] Loading 9 frames...
2025-11-03 16:10:40,119 [INFO] Combining 9 bias CCDData objects...
2025-11-03 16:10:40,168 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-11-03 16:10:52,957 [INFO] Master bias saved to kl4040.bias.comb.20250719.fits
2025-11-03 16:10:53,357 [INFO] Starting master dark creation...
2025-11-03 16:10:53,378 [INFO] Loading 27 frames...
2025-11-03 16:10:55,385 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-11-03 16:10:55,387 [INFO] Using master bias: kl4040.bias.comb.20250719.fits


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


2025-11-03 16:10:56,357 [INFO] Combining 9 darks for exptime 30.0s...
2025-11-03 16:10:56,433 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


...

2025-11-03 16:11:09,616 [INFO] Master dark saved to kl4040.dark.30s.comb.20250719.fits
2025-11-03 16:11:09,620 [INFO] Combining 9 darks for exptime 180.0s...
2025-11-03 16:11:09,705 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-11-03 16:11:23,073 [INFO] Master dark saved to kl4040.dark.180s.comb.20250719.fits
2025-11-03 16:11:23,077 [INFO] Combining 9 darks for exptime 300.0s...
2025-11-03 16:11:23,144 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-11-03 16:11:36,353 [INFO] Master dark saved to kl4040.dark.300s.comb.20250719.fits


In [4]:
logger.info("--- Step 3:  WCS solution update ---")
lv1 = solopy.Lv1(log_file=fpath_log_lv1)

# Collect all Lv0 fits files
all_lv0_fits = ccdproc.ImageFileCollection(LV0_subdir, glob_exclude="*.bz2", glob_include="*.fits")
science_frames = all_lv0_fits.files_filtered(IMAGETYP="LIGHT", include_path=True)

TMPDIR = DATADIR/"tmp"
TMPDIR.mkdir(parents=True, exist_ok=True)

if science_frames:
    for fpath_sci in science_frames:
        fpath_wcs = lv1.update_wcs(fpath_sci, TMPDIR, cache_directory=DATADIR/"astrometry_cache", return_fpath=True)
        print(fpath_wcs)

2025-11-03 16:58:12,353 [INFO] --- Step 3:  WCS solution update ---
2025-11-03 16:58:16,896 [WARNING] Astrometry.net solver failed for longexp_001_20250719065814.fits: Solver.__init__() got an unexpected keyword argument 'log_level'
2025-11-03 16:58:16,896 [WARNING] Skipping center coord calculation for longexp_001_20250719065814.fits (no WCS).
2025-11-03 16:58:16,922 [INFO] WCS updated: longexp_001_20250719065814.wcs.fits


../solo-data/tmp/longexp_001_20250719065814.wcs.fits


2025-11-03 16:58:21,037 [WARNING] Astrometry.net solver failed for longexp_001_20250719070221.fits: Solver.__init__() got an unexpected keyword argument 'log_level'
2025-11-03 16:58:21,037 [WARNING] Skipping center coord calculation for longexp_001_20250719070221.fits (no WCS).
2025-11-03 16:58:21,055 [INFO] WCS updated: longexp_001_20250719070221.wcs.fits


../solo-data/tmp/longexp_001_20250719070221.wcs.fits


KeyboardInterrupt: 

2025-11-03 16:35:58,063 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


2025-11-03 16:36:01,762 [INFO] downloading 3 files (169.19 MB)
2025-11-03 16:36:02,916 [INFO] downloading "http://data.astrometry.net/4100/index-4108.fits" to "/Users/bumhoo7/Desktop/solopy/notebook/astrometry_cache/4100/index-4108.fits"
2025-11-03 16:36:02,916 [INFO] downloading "http://data.astrometry.net/4100/index-4109.fits" to "/Users/bumhoo7/Desktop/solopy/notebook/astrometry_cache/4100/index-4109.fits"
2025-11-03 16:36:02,917 [INFO] downloading "http://data.astrometry.net/4100/index-4110.fits" to "/Users/bumhoo7/Desktop/solopy/notebook/astrometry_cache/4100/index-4110.fits"
2025-11-03 16:38:07,044 [INFO] loaded 3 index files
2025-11-03 16:38:07,046 [INFO] solve 1: start
2025-11-03 16:38:07,048 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4108.fits" (1 / 3)
2025-11-03 16:38:07,213 [INFO] solve 1: logodds=239.887, matches=42, conflicts=0, distractors=4, ra=270.196, dec=29.9716, scale=2.96297, index="4100/index-4108.fits"
2025-11-03 16:38:07,366 [INFO] solve 1: logodds=

../solo-data/tmp/longexp_001_20250719065814.wcs.fits
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


2025-11-03 16:38:13,381 [INFO] loaded 3 index files
2025-11-03 16:38:13,381 [INFO] solve 1: start
2025-11-03 16:38:13,383 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4108.fits" (1 / 3)
2025-11-03 16:38:13,555 [INFO] solve 1: logodds=238.681, matches=42, conflicts=0, distractors=4, ra=270.197, dec=29.972, scale=2.96288, index="4100/index-4108.fits"
2025-11-03 16:38:13,709 [INFO] solve 1: logodds=223.576, matches=42, conflicts=0, distractors=4, ra=270.196, dec=29.9683, scale=2.96347, index="4100/index-4108.fits"
2025-11-03 16:38:13,865 [INFO] solve 1: logodds=223.276, matches=42, conflicts=0, distractors=4, ra=270.195, dec=29.9691, scale=2.96366, index="4100/index-4108.fits"
2025-11-03 16:38:13,867 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4109.fits" (2 / 3)
2025-11-03 16:38:14,015 [INFO] solve 1: logodds=285.478, matches=41, conflicts=0, distractors=5, ra=270.194, dec=29.9708, scale=2.96246, index="4100/index-4109.fits"
2025-11-03 16:38:14,166 [INFO] solve 1:

...

2025-11-03 16:38:38,467 [INFO] loaded 3 index files
2025-11-03 16:38:38,468 [INFO] solve 1: start
2025-11-03 16:38:38,469 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4108.fits" (1 / 3)
2025-11-03 16:38:38,470 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4109.fits" (2 / 3)
2025-11-03 16:38:38,471 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4110.fits" (3 / 3)
2025-11-03 16:38:38,472 [INFO] solve 1: slice=[25, 50) (2 / 2), index="4100/index-4108.fits" (1 / 3)
2025-11-03 16:38:38,473 [INFO] solve 1: slice=[25, 50) (2 / 2), index="4100/index-4109.fits" (2 / 3)
2025-11-03 16:38:38,474 [INFO] solve 1: slice=[25, 50) (2 / 2), index="4100/index-4110.fits" (3 / 3)
2025-11-03 16:38:38,478 [WARNING] No WCS solution found for skyflat_001_20250719073146.fits.
2025-11-03 16:38:38,478 [WARNING] No WCS solution found for skyflat_001_20250719073146.fits.
2025-11-03 16:38:38,479 [WARNING] Skipping center coord calculation for skyflat_001_20250719073146.fits (no WCS).

../solo-data/tmp/skyflat_001_20250719073146.wcs.fits
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


2025-11-03 16:38:41,623 [INFO] loaded 3 index files
2025-11-03 16:38:41,624 [INFO] solve 1: start
2025-11-03 16:38:41,625 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4108.fits" (1 / 3)
2025-11-03 16:38:41,626 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4109.fits" (2 / 3)
2025-11-03 16:38:41,779 [INFO] solve 1: logodds=281.264, matches=43, conflicts=0, distractors=3, ra=270.125, dec=39.941, scale=2.9641, index="4100/index-4109.fits"
2025-11-03 16:38:41,935 [INFO] solve 1: logodds=285.128, matches=42, conflicts=0, distractors=4, ra=270.126, dec=39.9412, scale=2.9633, index="4100/index-4109.fits"
2025-11-03 16:38:42,087 [INFO] solve 1: logodds=272.733, matches=44, conflicts=0, distractors=2, ra=270.125, dec=39.9381, scale=2.97291, index="4100/index-4109.fits"
2025-11-03 16:38:42,089 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4110.fits" (3 / 3)
2025-11-03 16:38:42,237 [INFO] solve 1: logodds=324.462, matches=43, conflicts=0, distractors=3, ra=270.128

../solo-data/tmp/skyflat_001_20250719074006.wcs.fits
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


KeyboardInterrupt: 

In [ ]:



# Master bias
bias_files = all_lv0_fits.files_filtered(IMAGETYP="BIAS", include_path=True)
if bias_files:
    comb.make_mbias(bias_files, MASTERDIR)
    
# Master dark
dark_files = all_lv0_fits.files_filtered(IMAGETYP="DARK", include_path=True)
if dark_files:
    comb.make_mdark(dark_files, MASTERDIR, key_exptime='EXPTIME')